In [1]:
! pip install python-igraph
!pip install pyvis
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import re
import csv
import json
import rdflib
import gensim
import pandas as pd
from gensim import corpora
from pymongo import MongoClient
from nltk.corpus import stopwords
import plotly.graph_objects as go
from pyvis.network import Network
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [4]:
client = MongoClient("mongodb://localhost:27017/")
db = client['DataTails']
collection = db['Data']  
data_cursor = collection.find({})
DF = pd.DataFrame(list(data_cursor))
print(DF.head())

                        _id  type  \
0  66e965e698330736e0d693d5  None   
1  66e965e698330736e0d693d6  None   
2  66e965e698330736e0d693d7  None   
3  66e965e698330736e0d693d8  None   
4  66e965e698330736e0d693d9  None   

                                           postTitle postDesc  \
0  Adults(especially those over 30), how young do...      NaN   
1  What is a thing that your parents consider nor...      NaN   
2                 What is a smell that comforts you?      NaN   
3  When in history was it the best time to be a w...      NaN   
4    What's the worst way someone broke up with you?      NaN   

              postTime            authorName noOfUpvotes isNSFW  \
0  2024-08-06 01:02:35  Excellent-Studio7257        4068  False   
1  2024-08-06 01:47:22        Bigbumoffhappy        2073  False   
2  2024-08-05 22:21:53         bloomsmittenn        2188  False   
3  2024-08-06 03:32:59   More_food_please_77         778  False   
4  2024-08-05 21:01:39    ImpressiveWrap7363       

In [ ]:
lemmatizer = WordNetLemmatizer()
StopWords = set(stopwords.words('english'))
def Preprocessing(text):
    text = re.sub(r'\W', ' ', text)
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in StopWords]
    return ' '.join(tokens)

Cols = ['subReddit', 'postTitle', 'postDesc', 'postTime', 'authorName', 'noOfUpvotes', 'comments', 'noOfComments', 'postUrl']
DF = DF[Cols]
print(DF.isnull().sum())
DF['postDesc'].fillna('', inplace=True)
DF['noOfUpvotes'].fillna(0, inplace=True)
DF['postTime'] = pd.to_datetime(DF['postTime'], errors='coerce')
DF['comments'] = DF['comments'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
DF['postTitle'] = DF['postTitle'].apply(lambda x: Preprocessing(str(x)))
DF['postDesc'] = DF['postDesc'].apply(lambda x: Preprocessing(str(x)))
print(DF.head())
print(DF.isnull().sum())

In [ ]:
GroupedData = DF.groupby('subReddit')
SubReddits = defaultdict(dict)
for subreddit, group in GroupedData:
    print(f"Processing Subreddit: {subreddit}")
    group['Combined'] = group['postTitle'] + " " + group['postDesc']
    group['Tokens'] = group['Combined'].apply(lambda x: word_tokenize(x))
    dictionary = corpora.Dictionary(group['Tokens'])
    corpus = [dictionary.doc2bow(text) for text in group['Tokens']]
    LDA = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)
    SubReddits[subreddit]['lda_model'] = LDA
    SubReddits[subreddit]['dictionary'] = dictionary
    SubReddits[subreddit]['corpus'] = corpus
    group['topics'] = [LDA.get_document_topics(bow) for bow in corpus]
    DF.loc[group.index, 'topics'] = group['topics']

In [ ]:
print(DF.shape)
print(DF.describe())
print(DF.head())


In [5]:
lemmatizer = WordNetLemmatizer()
StopWords = set(stopwords.words('english'))
custom_stopwords = StopWords | {"make", "thing", "know", "http", "get", "want", "like", "would", "could", "you", "say","also","aita","com","www","made","ago","day","000"}
def Preprocessing(text):
    text = re.sub(r'\W', ' ', text)  # Remove non-alphanumeric characters
    text = re.sub(r'\b\w{1,2}\b', '', text)  # Remove short words
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in custom_stopwords and len(word) > 2]
    return tokens

Cols = ['subReddit', 'postTitle', 'postDesc', 'postTime', 'authorName', 'noOfUpvotes', 'comments', 'noOfComments', 'postUrl','imageUrl','isNSFW']
DF = DF[Cols]
print(DF.isnull().sum())


DF['subReddit'] = DF['subReddit'].fillna('Unknown_SubReddit')
DF['authorName'] = DF['authorName'].fillna('Unknown_Author')
DF['postTitle'] = DF['postTitle'].fillna('Untitled')
DF['postUrl'] = DF['postUrl'].fillna('http://example.com/NOPOST.png')
DF['imageUrl'] = DF['imageUrl'].fillna('http://example.com/NOImage.png')
DF['isNSFW'] = DF['isNSFW'].fillna(False)
DF['postDesc'].fillna('', inplace=True)
DF['noOfUpvotes'].fillna(0, inplace=True)
DF['noOfComments'].fillna(0, inplace=True)
DF['postTime'] = pd.to_datetime(DF['postTime'], errors='coerce')
DF['comments'] = DF['comments'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
DF['postTitle'] = DF['postTitle'].apply(lambda x: Preprocessing(str(x)))
DF['postDesc'] = DF['postDesc'].apply(lambda x: Preprocessing(str(x)))
DF['isNSFW'] = DF['isNSFW'].astype(bool)
print(DF.head())
print(DF.isnull().sum())

# Initialize graph and namespaces
g = rdflib.Graph()
SIOC = rdflib.Namespace("http://rdfs.org/sioc/ns#")
DCMI = rdflib.Namespace("http://purl.org/dc/elements/1.1/")
FOAF = rdflib.Namespace("http://xmlns.com/foaf/0.1/")
g.bind("sioc", SIOC)
g.bind("dc", DCMI)
g.bind("foaf", FOAF)

# Function to add posts to the graph
def add_post_to_graph(row, index, topic_uri):
    post_uri = rdflib.URIRef(f"http://reddit.com/post{index}")
    subreddit_uri = rdflib.URIRef(f"http://reddit.com/subreddit/{row['subReddit']}")
    author_uri = rdflib.URIRef(f"http://reddit.com/user/{row['authorName']}")
    
    g.add((post_uri, rdflib.RDF.type, SIOC.Post))
    g.add((post_uri, DCMI.title, rdflib.Literal(row['postTitle'])))
    g.add((post_uri, DCMI.description, rdflib.Literal(row['postDesc'])))
    g.add((post_uri, DCMI.date, rdflib.Literal(row['postTime'])))
    g.add((post_uri, SIOC.num_replies, rdflib.Literal(row['noOfUpvotes'])))
    g.add((post_uri, SIOC.link, rdflib.URIRef(row['postUrl'])))
    
    g.add((subreddit_uri, rdflib.RDF.type, SIOC.Container))
    g.add((subreddit_uri, SIOC.has_post, post_uri))
    g.add((post_uri, SIOC.Container, subreddit_uri))

    g.add((author_uri, rdflib.RDF.type, FOAF.Person))
    g.add((author_uri, FOAF.name, rdflib.Literal(row['authorName'])))
    g.add((post_uri, SIOC.has_creator, author_uri))

    comment_uri = rdflib.URIRef(f"http://reddit.com/comment/{index}")
    g.add((comment_uri, rdflib.RDF.type, SIOC.Comment))
    g.add((comment_uri, DCMI.title, rdflib.Literal(row['comments'])))
    g.add((post_uri, SIOC.has_reply, comment_uri))

    g.add((topic_uri, SIOC.has_post, post_uri))

# Group by subreddit and process each one for topic modeling
GroupedData = DF.groupby('subReddit')
all_topics = defaultdict(set)
for subreddit, group in GroupedData:
    print(f"Processing Subreddit: {subreddit}")
    group['Combined'] = group['postTitle'] + group['postDesc']
    
    group['Tokens'] = group['Combined'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x if word not in custom_stopwords])
    
    dictionary = corpora.Dictionary(group['Tokens'])
    corpus = [dictionary.doc2bow(text) for text in group['Tokens']]

    LDA = gensim.models.LdaModel(
        corpus,
        num_topics=5,
        id2word=dictionary,
        passes=15,
        iterations=100,
        random_state=42
    )
    
    for idx, topic in LDA.print_topics(num_words=5):
        topic_words = topic.split(" + ")
        unique_words = {word.split("*")[1].strip('"') for word in topic_words}
        all_topics[subreddit].update(unique_words)

# Save RDF graph in Turtle and JSON-LD formats
g.serialize('D:/FYP/Github/data-tails/Backend/Ontologies/KG1.ttl', format='turtle')
json_ld = g.serialize(format='json-ld', indent=4)
with open("D:/FYP/Github/data-tails/Backend/Ontologies/KG1.json", "w", encoding="utf-8") as f:
    f.write(json_ld)


subReddit           2
postTitle           0
postDesc        15607
postTime            0
authorName        587
noOfUpvotes         0
comments            0
noOfComments        2
postUrl             2
imageUrl            3
isNSFW              1
dtype: int64
   subReddit                                          postTitle postDesc  \
0  AskReddit                   [adult, especially, young, seem]       []   
1  AskReddit  [parent, consider, normal, consider, normal, a...       []   
2  AskReddit                                   [smell, comfort]       []   
3  AskReddit                       [history, best, time, woman]       []   
4  AskReddit                       [worst, way, someone, broke]       []   

             postTime            authorName noOfUpvotes  \
0 2024-08-06 01:02:35  Excellent-Studio7257        4068   
1 2024-08-06 01:47:22        Bigbumoffhappy        2073   
2 2024-08-05 22:21:53         bloomsmittenn        2188   
3 2024-08-06 03:32:59   More_food_please_77         

In [6]:
# Save subreddits and topics to Topics.csv
with open("D:/FYP/Github/data-tails/Backend/Ontologies/Topics.csv", mode='w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["subreddit", "topics"])
    for subreddit, topics in all_topics.items():
        writer.writerow([subreddit, ", ".join(sorted(topics))])

print("Subreddits and topics have been saved to Topics.csv.")



# List of subreddits to include in the new CSV
selected_subreddits = [
    "todayilearned", "news", "mildlyinteresting", "AskReddit", "Damnthatsinteresting", 
    "showerthoughts", "wallstreetbets", "NatureIsFuckingLit", "LifeProTips", "space", 
    "MovieDetails", "tifu", "futurology", "Music", "europe", "unpopularopinion", 
    "pakistan", "islamabad", "lahore", "karachi"
]

# Read the existing topics CSV
topics_df = pd.read_csv("D:/FYP/Github/data-tails/Backend/Ontologies/Topics.csv")

# Filter rows where 'subreddit' is in the selected subreddits list
filtered_topics_df = topics_df[topics_df['subreddit'].isin(selected_subreddits)]

# Save the filtered data to a new CSV file
filtered_topics_csv_path = "D:/FYP/Github/data-tails/Backend/Ontologies/FTopics.csv"
filtered_topics_df.to_csv(filtered_topics_csv_path, index=False)

print(f"Filtered topics saved to {filtered_topics_csv_path}")

Subreddits and topics have been saved to Topics.csv.
Filtered topics saved to D:/FYP/Github/data-tails/Backend/Ontologies/FTopics.csv


## GraphRag

In [ ]:
import rdflib
from rdflib import Graph, Namespace, Literal
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import subprocess
import re
from isodate import parse_datetime, ISO8601Error

# Namespaces
SIOC = Namespace("http://rdfs.org/sioc/ns#")
DCMI = Namespace("http://purl.org/dc/elements/1.1/")
FOAF = Namespace("http://xmlns.com/foaf/0.1/")
reddit = Namespace("http://reddit.com/")

# Preprocessing
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    tokens = word_tokenize(text.lower())
    return [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 2]

def clean_dates(graph):
    for s, p, o in graph.triples((None, DCMI.date, None)):
        if isinstance(o, Literal):
            try:
                parse_datetime(str(o))  # Validate
            except ISO8601Error:
                graph.remove((s, p, o))
                graph.add((s, p, Literal("1970-01-01T00:00:00", datatype=SIOC.date)))

def retrieve_expanded_context(graph, user_query, depth=2):
    sparql_query = f"""
    SELECT ?subject ?predicate ?object
    WHERE {{
        ?subject ?predicate ?object .
        OPTIONAL {{ ?subject <{DCMI.title}> ?title }}
        OPTIONAL {{ ?subject <{DCMI.description}> ?description }}
        FILTER(
            CONTAINS(LCASE(STR(?title)), LCASE("{user_query}")) || 
            CONTAINS(LCASE(STR(?description)), LCASE("{user_query}"))
        )
    }}
    LIMIT {depth * 10}
    """
    results = graph.query(sparql_query)
    return "\n".join([f"{s} --[{p}]--> {o}" for s, p, o in results])

def query_wizardlm2(context, user_query):
    prompt = f"Context:\n{context}\n\nQuestion:\n{user_query}\n"
    try:
        result = subprocess.run(
            ["ollama", "run", "wizardlm2"],
            input=prompt.encode("utf-8"),  # Encode input
            capture_output=True
        )
        if result.returncode != 0:
            raise Exception(result.stderr.decode("utf-8"))
        return result.stdout.decode("utf-8").strip()
    except Exception as e:
        return f"Error querying WizardLM2: {str(e)}"

if __name__ == "__main__":
    kg_file = "./KG.ttl"
    kg = Graph()
    kg.parse(kg_file, format="turtle")
    clean_dates(kg)

    user_query = "I have OCD, neeed help?"
    context = retrieve_expanded_context(kg, user_query, depth=3)
    print(f"Retrieved Context:\n{context}")

    response = query_wizardlm2(context, user_query)
    print(f"\nLLM Response:\n{response}")


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x000001F35584A9E0>
Traceback (most recent call last):
  File "c:\Users\97156\AppData\Local\Programs\Python\Python310\lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "c:\Users\97156\AppData\Local\Programs\Python\Python310\lib\site-packages\isodate\isodatetime.py", line 55, in parse_datetime
    tmpdate = parse_date(datestring)
  File "c:\Users\97156\AppData\Local\Programs\Python\Python310\lib\site-packages\isodate\isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: 'Na'


Retrieved Context:

